In [3]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [4]:
train=pd.read_csv('trainset.csv')


In [5]:
test=pd.read_csv('testset.csv')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Columns: 1614 entries, cls to mZagreb2
dtypes: float64(1276), int64(338)
memory usage: 98.5 MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 1614 entries, cls to mZagreb2
dtypes: float64(1273), int64(341)
memory usage: 24.6 MB


In [9]:
train[:5]

,cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,1,NaN,NaN,0,0,25.001104,2.501652,4.923679,25.001104,1.250055,...,9.918524,67.526731,284.075684,8.877365,764,32,100,119,7.777778,4.666667
1,1,NaN,NaN,0,0,20.274271,2.409184,4.779725,20.274271,1.267142,...,9.636588,61.986524,226.038985,9.827782,429,23,82,96,6.166667,3.583333
2,1,NaN,NaN,0,0,23.679070,2.528665,4.931112,23.679070,1.392886,...,10.022647,65.026165,223.074562,8.579791,461,29,98,121,3.416667,3.638889
3,1,NaN,NaN,0,0,27.774585,2.557762,4.967131,27.774585,1.388729,...,10.250228,70.773493,266.069142,8.868971,735,35,120,150,3.888889,4.166667
4,1,NaN,NaN,0,0,39.613943,2.477563,4.835703,39.613943,1.320465,...,10.233043,79.537169,415.183067,7.548783,2674,45,156,182,8.500000,6.694444


In [10]:
import torch
import torch.nn as nn

In [12]:
class Autoencoder(nn.Module):
    
    def __init__(self, input_size, encoding_dim):
        
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 800),
            nn.ReLU(),
            nn.Linear(800,400),
            nn.ReLU(),
            nn.Linear(400,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,16)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(16,32),
            nn.ReLU(),
        )

    def forward(self, x):
        
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded

Request info on why ABC, ABCGG,nAcid,nBase are 0

In [8]:
class cnn(nn.Module):
    
    def __init__(self,input_size,num_classes):
        drop_per=0.05
        super(cnn,self)
        
        # 1st convolution
        self.conv1d1 = nn.Conv1d(in_channels=input_size,out_channels=800,kernel_size=3)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=3)
        self.dropout1 = nn.Dropout(p=drop_per)
        
        # 2nd convolution 
        self.conv1d2 = nn.Conv1d(in_channels=64,out_channels=,kernel_size=3)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=3)
        self.dropout2 = nn.Dropout(p=drop_per)

SyntaxError: invalid decimal literal (4279969320.py, line 4)